# ENGR 510: Introduction to Non-Neural Network ML

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

# For importing the MNIST data set:
from sklearn.datasets import fetch_openml

# Scikit-learn PCA model:
from sklearn.decomposition import PCA

# Scikit-learn LDA model:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Scikit-learn SVM classifier:
from sklearn.svm import LinearSVC

# Scikit-learn decision tree classifier:
from sklearn.tree import DecisionTreeClassifier

First, let us import and visualize the MNIST data set by running the following code block.

**Please run the cell below. DO NOT edit this cell.**

In [2]:
# Load the MNIST data:
mnist = fetch_openml("mnist_784", parser="auto")
X = np.array(mnist.data) / 255.0  # Scale the data to [0, 1]
y = np.array(mnist.target)

# Print some data information:
print("MNIST data loaded succesfully!")
print(f"Image data: X.shape = {X.shape}")
print(f"Label data: y.shape = {y.shape}")

MNIST data loaded succesfully!
Image data: X.shape = (70000, 784)
Label data: y.shape = (70000,)


In [3]:
# # Plot some of the MNIST images.
# plt.figure(figsize=(14, 7))
# for i in range(15):
#     plt.subplot(3, 5, i + 1)
#     plt.title(f"Image {i + 1}\nLabel = {y[i]}")
#     plt.imshow(X[i, :].reshape(28, 28), cmap='binary')
#     plt.colorbar()
# plt.tight_layout()
# plt.show()

In [4]:
# Mean center the data for PCA.
x_mean = np.mean(X, axis=0)
X = X - x_mean

Notice that there are 70,000 images total, with each image being 28 x 28 pixels, or 784 pixels total.

Each image comes with a corresponding label, i.e. a number that indicates what digit is inside the image.

## Part 1: Dimensionality Reduction with PCA

### 1.1 Perform SVD

In [5]:
### TODO: Perform SVD analysis.
U, S, Vh = np.linalg.svd(X, full_matrices=False)
V = Vh.T
print(f"SVD: U.shape = {U.shape}")
print(f"SVD: S.shape = {S.shape}")
print(f"SVD: V.shape = {V.shape}")

SVD: U.shape = (70000, 784)
SVD: S.shape = (784,)
SVD: V.shape = (784, 784)


In [6]:
### TODO: Save the first five dominant modes.
A1 = V[:, 0:5]
print(f"A1.shape = {A1.shape}")

A1.shape = (784, 5)


In [7]:
# # # OPTIONAL: Plot columns of V.
# plt.figure(figsize=(14, 2))
# for i, v in enumerate(A1.T):
#     plt.subplot(1, 5, i + 1)
#     plt.title(f"$v_{i + 1}$")
#     plt.imshow(v.reshape(28, 28), cmap='RdGy', vmin = -0.2, vmax = 0.2)
#     plt.colorbar()
# plt.tight_layout()
# plt.show()

### 1.2 Reconstruct First MNIST Image

In [8]:
### TODO: Reconstruct the first MNIST image.
r_values = [5, 50, 100, 300, 784]
x = X[0]
xr = np.zeros((len(r_values), *x.shape))
for i, r in enumerate(r_values):
    Vr = V[:, 0:r]
    xr[i] = Vr @ Vr.T @ x

xr = xr + x_mean

In [9]:
# plt.figure(figsize=(14, 2))
# for i, x in enumerate(xr):
#     im = xr[i].reshape(28, 28)
#     scale_max = np.max(abs(im))
#     scale_min = -scale_max
#     plt.subplot(1, len(r_values), i+1)
#     plt.title(f"$x_{{{r_values[i]}}}$")
#     plt.imshow(im, cmap='RdGy', vmin=scale_min, vmax=scale_max)
#     plt.colorbar()
# plt.tight_layout()
# plt.savefig("1.2fig1.png", dpi=300)
# plt.show()

### 1.3 Examine Singular Value Spectrum

In [10]:
# ### TODO: Plot the singular values.
# N = 100
# plt.figure(figsize=(14, 4))
# plt.subplot()
# plt.xlabel("$r$")
# plt.ylabel("$\sigma_r$")
# plt.bar(np.arange(1, len(S[:N])+1), S[:N], width=0.5)
# plt.gca().xaxis.set_major_locator(MultipleLocator(5))
# plt.xlim(1e-6, N+1)
# plt.tight_layout()
# plt.savefig("1.3fig1.png", dpi=300)
# plt.show()

In [11]:
eig_sum = np.sum(S*S)
for i in np.arange(1, len(S)+1):
    variance = np.sum(S[:i]*S[:i]) / eig_sum
    if variance >= 0.99:
        print(f"r={i} retains {variance * 100}% of variance")
        r99 = i
        break

### TODO: Compute the rank truncation r.
A2 = r99
print(f"type(A2) = {type(A2)}")

r=331 retains 99.0055029257276% of variance
type(A2) = <class 'numpy.int64'>


In [12]:
# # Plot rank truncation
# plt.figure(figsize=(14, 4))
# plt.subplot()
# plt.title(f"Singular Values")
# plt.xlabel("$r$")
# plt.ylabel("$\sigma_r$")
# plt.bar(np.arange(1, len(S)+1), S, width=1)
# plt.axvline(r99, c='r')
# plt.xlim(1e-6, len(S)+1)
# plt.tight_layout()
# plt.show()

### 1.4 PCA Space Visualization

In [13]:
r = 350

### TODO: Project data onto the 350 leading V modes.
X_pca = X @ V[:, 0:r]

In [14]:
# ### TODO: Plot projected data in 3-D.
# num_plot = 500
# elevs = [30]
# azims = [45]
# for i, (elev, azim) in enumerate(zip(elevs, azims)):
#     fig = plt.figure()
#     ax = fig.add_subplot(projection="3d")
#     sc = ax.scatter(
#         X_pca[:num_plot, 0],
#         X_pca[:num_plot, 1],
#         X_pca[:num_plot, 2],
#         c=y[:num_plot].astype(int),
#         cmap="tab10",
#         marker=".",
#     )
#     ax.view_init(elev=elev, azim=azim)
#     ax.set_xlabel('$\sigma_1$')
#     ax.set_ylabel('$\sigma_2$')
#     ax.set_zlabel('$\sigma_3$')
#     plt.colorbar(sc)
#     plt.tight_layout()
#     plt.savefig(f"1.4fig{i+1}.png", dpi=300)
# plt.show()

## Part 2: MNIST Digit Classification

First, obtain the rank $r=350$ PCA-projected data.

**Please run the cell below. DO NOT edit this cell.**

In [15]:
# Use Scikit-learn PCA model to reduce the data.
pca = PCA(n_components=350)
X_pca = pca.fit_transform(X)

print(f"PCA image data: X_pca.shape = {X_pca.shape}")
print(f"Label data: y.shape = {y.shape}")

PCA image data: X_pca.shape = (70000, 350)
Label data: y.shape = (70000,)


The following code has been provided to help you with digit extraction and train / test splitting.

Feel free to use or not use this code, but please read the documentation carefully!

In [16]:
def get_digit_data(
    X: np.ndarray,
    y: np.ndarray,
    digit_list: list,
    n_test: int,
):
    """
    Helper function that takes the given data+labels, extracts the data+labels
    containing the desired digits, and returns a training / testing data split.

    Args:
        X = (n_samples, n_features) np.ndarray of data.
        y = (n_samples,) np.ndarray of corresponding labels.
        digit_list = list of desired MNIST digits to filter out.
        n_test = integer number of digits to take for the test set.
            The first n_test digits are always taken for the test set.
    Returns:
        1. (n_train, n_features) np.ndarray of training data.
        2. (n_train,) np.ndarray of training data labels.
        3. (n_test, n_features) np.ndarray of test data.
        4. (n_test,) np.ndarray of test data labels.
    """
    inds_test = np.array([], dtype=int)
    inds_train = np.array([], dtype=int)

    for digit in digit_list:
        digit_inds = np.where(y.astype(int) == digit)[0]
        inds_test = np.union1d(inds_test, digit_inds[:n_test])
        inds_train = np.union1d(inds_train, digit_inds[n_test:])

    return X[inds_train], y[inds_train], X[inds_test], y[inds_test]

# EXAMPLE: How to form a training / testing data split on the PCA projected data
# using the digits 0, 9 while setting the first 100 samples aside per digit for testing.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, digit_list=[0, 9], n_test=100)
print(f"Training data: X_train.shape = {X_train.shape}")
print(f"Training labels: y_train.shape = {y_train.shape} y_train = {y_train}")
print(f"Testing data: X_test.shape = {X_test.shape}")
print(f"Testing labels: y_test.shape = {y_test.shape} y_test[:10] = {y_test[:10]}")

Training data: X_train.shape = (13661, 350)
Training labels: y_train.shape = (13661,) y_train = ['9' '9' '0' ... '0' '9' '0']
Testing data: X_test.shape = (200, 350)
Testing labels: y_test.shape = (200,) y_test[:10] = ['0' '9' '9' '0' '9' '9' '0' '0' '9' '9']


In [17]:
### TODO: Implement classification accuracy percentage.
def compute_accuracy(y, y_true):
    """
    Args:
        y = (n_samples,) np.ndarray of computed labels.
        y_true = (n_samples,) np.ndarray of true labels.
    Returns:
        Classification accuracy percentage as a float.
    """
    y = np.array(y)
    y_true = np.array(y_true)
    return np.sum(y == y_true)/len(y)*100

### 2.1 LDA with 3 and 4

In [18]:
# TODO: Fit an LDA model using the digits 3, 4.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, digit_list=[3, 4], n_test=100)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

y_train_predict = lda.predict(X_train)
y_test_predict = lda.predict(X_test)

train_accuracy = compute_accuracy(y_train_predict, y_train)
test_accuracy = compute_accuracy(y_test_predict, y_test)
print(f"Train accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")

A3 = np.array([train_accuracy, test_accuracy])
print(f"A3.shape = {A3.shape}")

Train accuracy: 99.3%
Test accuracy: 98.0%
A3.shape = (2,)


### 2.2 LDA with 3, 5, and 9

In [19]:
# TODO: Fit an LDA model using the digits 3, 5, 9.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, digit_list=[3, 5, 9], n_test=100)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

y_train_predict = lda.predict(X_train)
y_test_predict = lda.predict(X_test)

train_accuracy = compute_accuracy(y_train_predict, y_train)
test_accuracy = compute_accuracy(y_test_predict, y_test)
print(f"Train accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")

A4 = np.array([train_accuracy, test_accuracy])
print(f"A4.shape = {A4.shape}")

Train accuracy: 95.3%
Test accuracy: 93.7%
A4.shape = (2,)


### 2.3 Hardest LDA Pair

In [20]:
# TODO: Fit an LDA model using all digit pairs.
# X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, digit_list=np.arange(0, 10), n_test=100)

# lda = LinearDiscriminantAnalysis()
# lda.fit(X_train, y_train)

# y_train_predict = lda.predict(X_train)
# y_test_predict = lda.predict(X_test)

# train_accuracy = compute_accuracy(y_train_predict, y_train)
# test_accuracy = compute_accuracy(y_test_predict, y_test)
# print(f"Train accuracy: {train_accuracy:.1f}%")
# print(f"Test accuracy: {test_accuracy:.1f}%")

In [21]:
# function to perform LDA of given digits
def lda_process(digit_list, n_test):
    X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, digit_list, n_test)
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    y_train_predict = lda.predict(X_train)
    y_test_predict = lda.predict(X_test)
    train_accuracy = compute_accuracy(y_train_predict, y_train)
    test_accuracy = compute_accuracy(y_test_predict, y_test)
    return lda, train_accuracy, test_accuracy

In [22]:
# LDA for all pairs
n_test = 500
pair_train_accuracy = np.zeros((10, 10))
pair_test_accuracy = np.zeros((10, 10))
digits = np.arange(0, 10)
for i, j in zip(*np.triu_indices(len(digits), 1)):
    _, _train_accuracy, _test_accuracy = lda_process([digits[i], digits[j]], n_test)
    pair_train_accuracy[i, j] = _train_accuracy
    pair_test_accuracy[i, j] = _test_accuracy

In [23]:
# # Plot pairs
# fig = plt.figure()
# ax = plt.subplot()
# im = ax.imshow((pair_test_accuracy-95).clip(min=0))
# ax.set_xticks(digits, labels=digits)
# ax.set_yticks(digits, labels=digits)
# plt.colorbar(im)
# plt.title("Accuracy (over 95%)")
# plt.show()

# print(pair_test_accuracy)

In [24]:
# Find hardest pair
acc_flattened = pair_test_accuracy.flatten()
nonzero_idxs = np.nonzero(acc_flattened)[0]
idx_hardest = nonzero_idxs[np.argmin(acc_flattened[nonzero_idxs])]
i, j = np.unravel_index(idx_hardest, pair_test_accuracy.shape)
hardest = [i, j]
hardest_digits = np.sort(digits[hardest])
hardest_accuracy = np.array([pair_train_accuracy[hardest[0], hardest[1]], pair_test_accuracy[hardest[0], hardest[1]]])

print(f"Hardest pair: {hardest_digits[0]} and {hardest_digits[1]} with training accuracy {hardest_accuracy[0]:.1f}% and testing accuracy {hardest_accuracy[1]:.1f}%")

# TODO: Plot the hardest digit pair.
A5 = hardest_digits
A6 = hardest_accuracy
print(f"A5 = {A5}")
print(f"A6 = {A6}")

Hardest pair: 3 and 5 with training accuracy 96.1% and testing accuracy 95.5%
A5 = [3 5]
A6 = [96.12172796 95.5       ]


### 2.4 Easiest LDA Pair

In [25]:
# Find easiest pair
idx_easiest = nonzero_idxs[np.argmax(acc_flattened[nonzero_idxs])]
i, j = np.unravel_index(idx_easiest, pair_test_accuracy.shape)
easiest = [i, j]
easiest_digits = np.sort(digits[easiest])
easiest_accuracy = np.array([pair_train_accuracy[easiest[0], easiest[1]], pair_test_accuracy[easiest[0], easiest[1]]])

print(f"Easiest pair: {easiest_digits[0]} and {easiest_digits[1]} with training accuracy {easiest_accuracy[0]:.1f}% and testing accuracy {easiest_accuracy[1]:.1f}%")

# TODO: Plot the easiest digit pair.
A7 = easiest_digits
A8 = easiest_accuracy
print(f"A7 = {A7}")
print(f"A8 = {A8}")

Easiest pair: 6 and 7 with training accuracy 99.8% and testing accuracy 99.7%
A7 = [6 7]
A8 = [99.80256663 99.7       ]


### 2.5 Hardest and Easiest Pair PCA Space Visualization

In [26]:
# ### TODO: Plot projected data in 3-D.
# X_hardest, y_hardest, _, _ = get_digit_data(X_pca, y, hardest_digits, 0)
# X_easiest, y_easiest, _, _ = get_digit_data(X_pca, y, easiest_digits, 0)

# num_plot = 500
# elevs = [60]
# azims = [45]
# colors = ['tab:blue', 'tab:orange']
# for i, (_X, _y, _digits) in enumerate(zip([X_hardest, X_easiest], [y_hardest, y_easiest], [hardest_digits, easiest_digits])):
#     _y = np.float64(_y)
#     for j, (elev, azim) in enumerate(zip(elevs, azims)):
#         fig = plt.figure()
#         ax = plt.subplot(projection="3d")
#         for k in range(2):
#             mask = _y[:num_plot]==_digits[k]
#             sc = ax.scatter(
#                 _X[:num_plot][mask, 0],
#                 _X[:num_plot][mask, 1],
#                 _X[:num_plot][mask, 2],
#                 c=[colors[k]]*np.count_nonzero(mask),
#                 marker=".",
#                 label=str(_digits[k])
#             )
#         ax.view_init(elev=elev, azim=azim)
#         ax.set_xlabel('$\sigma_1$')
#         ax.set_ylabel('$\sigma_2$')
#         ax.set_zlabel('$\sigma_3$')
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f"2.5fig{i+1}.png", dpi=300)
# plt.show()

### 2.6 SVM

In [27]:
# TODO: Fit an SVM model using the digits 7, 9.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, [7, 9], 500)

svm = LinearSVC()
svm.fit(X_train, y_train)

y_train_predict = svm.predict(X_train)
y_test_predict = svm.predict(X_test)

train_accuracy = compute_accuracy(y_train_predict, y_train)
test_accuracy = compute_accuracy(y_test_predict, y_test)
print(f"Train accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")

A9 = np.array([train_accuracy, test_accuracy])
print(f"A9 = {A9}")

Train accuracy: 97.0%
Test accuracy: 96.4%
A9 = [96.9662667 96.4      ]


In [28]:
# TODO: Fit an SVM model using the digits 1, 6.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, [1, 6], 500)

svm = LinearSVC()
svm.fit(X_train, y_train)

y_train_predict = svm.predict(X_train)
y_test_predict = svm.predict(X_test)

train_accuracy = compute_accuracy(y_train_predict, y_train)
test_accuracy = compute_accuracy(y_test_predict, y_test)
print(f"Train accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")

A10 = np.array([train_accuracy, test_accuracy])
print(f"A10 = {A10}")

Train accuracy: 100.0%
Test accuracy: 100.0%
A10 = [100. 100.]


### 2.7 Decision Tree

In [29]:
# TODO: Fit a decision tree using the digits 4, 9.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, [4, 9], 500)

tree = DecisionTreeClassifier(random_state=1234)
tree.fit(X_train, y_train)

y_train_predict = tree.predict(X_train)
y_test_predict = tree.predict(X_test)

train_accuracy = compute_accuracy(y_train_predict, y_train)
test_accuracy = compute_accuracy(y_test_predict, y_test)
print(f"Train accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")

A11 = np.array([train_accuracy, test_accuracy])
print(f"A11 = {A11}")

Train accuracy: 100.0%
Test accuracy: 88.8%
A11 = [100.   88.8]


In [30]:
# TODO: Fit a decision tree using the digits 0, 1.
X_train, y_train, X_test, y_test = get_digit_data(X_pca, y, [0, 1], 500)

tree = DecisionTreeClassifier(random_state=1234)
tree.fit(X_train, y_train)

y_train_predict = tree.predict(X_train)
y_test_predict = tree.predict(X_test)

train_accuracy = compute_accuracy(y_train_predict, y_train)
test_accuracy = compute_accuracy(y_test_predict, y_test)
print(f"Train accuracy: {train_accuracy:.1f}%")
print(f"Test accuracy: {test_accuracy:.1f}%")

A12 = np.array([train_accuracy, test_accuracy])
print(f"A12 = {A12}")

Train accuracy: 100.0%
Test accuracy: 99.6%
A12 = [100.   99.6]
